In [1]:
#%pip install llama-index
#%pip install fsspec
#%pip install gcsfs
#%pip install openpyxl

In [2]:
from llama_index import GPTTreeIndex, SimpleDirectoryReader
from llama_index.llm_predictor import HuggingFaceLLMPredictor
from IPython.display import Markdown, display

import os

In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
data_dir = "../test data/sample_articles"

In [5]:
from llama_index.prompts.prompts import SimpleInputPrompt

system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
""" 

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [6]:
import torch


hf_predictor = HuggingFaceLLMPredictor(
    max_input_size=4096, 
    max_new_tokens=2048,
    temperature=0,
    do_sample=False,
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="stabilityai/stablelm-tuned-alpha-7b",
    model_name="stabilityai/stablelm-tuned-alpha-7b",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
from llama_index import ServiceContext


service_context = ServiceContext.from_defaults(chunk_size_limit=1024, llm_predictor=hf_predictor)

In [10]:
documents = SimpleDirectoryReader(data_dir).load_data()
new_index = GPTTreeIndex.from_documents(documents, service_context=service_context)

INFO:llama_index.indices.common_tree.base:> Building index from nodes: 2 chunks
> Building index from nodes: 2 chunks


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 6837 tokens
> [build_index_from_nodes] Total LLM token usage: 6837 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
> [build_index_from_nodes] Total embedding token usage: 0 tokens


In [11]:
query_engine = new_index.as_query_engine()
response = query_engine.query("Which companies made drugs to help alzheimer disease?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 0] Selected node: [1]/[1]
>[Level 0] Selected node: [1]/[1]


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 1] Selected node: [1]/[1]
>[Level 1] Selected node: [1]/[1]
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 5744 tokens
> [retrieve] Total LLM token usage: 5744 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 0 tokens
> [retrieve] Total embedding token usage: 0 tokens


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 3055 tokens
> [get_response] Total LLM token usage: 3055 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


In [12]:
print(response.response)

Pfizer is a pharmaceutical company that specializes in developing drugs to treat Alzheimer's disease. The company reported first-quarter revenue and adjusted earnings that topped Wall Street's expectations, despite a decline in sales driven by the lower demand for the company's Covid vaccine. Pfizer's net income of $5.54 billion, or 97 cents per share, fell from $7.86 billion, or $1.37 per share, during the year-earlier quarter.

Pfizer's CEO, Albert Bourla, said during the earnings call that the company expects 2023 to be a "transition year" for Covid sales as the U.S. pivots to the commercial market for Covid products.

Pfizer CEO, Albert Bourla, said during the earnings call that the company expects 2023 to be a "transition year" for Covid sales as the U.S. pivots to the commercial market for Covid products.

Pfizer CEO, Albert Bourla, said during the earnings call that the company expects 2023 to be a "transition year" for Covid sales as the U.S. pivots to the commercial market for